In [3]:
import os
import numpy as np  
import pandas as pd
import keras as K
import matplotlib.pyplot as plt
from keras.models import Model
from keras.models import Sequential
from sklearn.metrics import accuracy_score, matthews_corrcoef, f1_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

def bayes_classifier(train_x, train_y):
    model = MultinomialNB()
    model.fit(train_x, train_y)
    return model

def knn_classifier(train_x, train_y):   
    model = KNeighborsClassifier(n_neighbors=2)
    model.fit(train_x, train_y)
    return model

def random_forest_classifier(train_x, train_y):
    model = RandomForestClassifier(n_estimators=10)
    model.fit(train_x, train_y)
    return model

def decision_tree_classifier(train_x, train_y):
    model = tree.DecisionTreeClassifier()
    model.fit(train_x, train_y)
    return model

def gradient_boosting_classifier(train_x, train_y):
    model = GradientBoostingClassifier(n_estimators=10)
#    model = XGBClassifier(n_estimators=10)
    model.fit(train_x, train_y)
    return model

def svm_classifier(train_x, train_y):
    model = SVC(kernel='rbf', probability=True)
    model.fit(train_x, train_y)
    return model

def normdata(data):  
    min_x = min(data)
    max_x = max(data)
    norm = max_x - min_x
    data = (data - min_x)/norm
    return data

def data_zero(data):  
    num = len(data)
    for i in range(num):
        if(data[i]<0.1):
            data[i]=0
    return data

def main_eval(data):
    dataset = normdata(data)
    dataset = data_zero(dataset)    
    dataset = np.reshape(dataset,(1,len(dataset)))        
    dataset = dataset.astype(np.float32)       
    return dataset


In [4]:
from sklearn.metrics import confusion_matrix
classifiers = {'NB':bayes_classifier,   
                   'KNN' :knn_classifier,                      
                   'RF':random_forest_classifier,  
                   'DT':decision_tree_classifier,  
                   'SVM':svm_classifier,                    
                   'GBDT':gradient_boosting_classifier,          
    }  

In [5]:
f = open('c1.txt','w')
num = 100
f1_p = np.zeros(6)
f1 = np.zeros(6)
recall_p = np.zeros(6)
recall = np.zeros(6)

f1 = open('pred_1.csv','w')
path_dir = 'lbs_22.csv'
with open(path_dir,'r',encoding='utf-8') as fs:
    lines = fs.readlines()
fs.close()
print(lines[0],end='',file=f1)


lb_dataset = pd.read_csv('lbs.csv')  
img_list = np.array(lb_dataset)
np.random.seed(0)
np.random.shuffle(img_list)
lb_data = img_list[:,5:] 
lb_target = img_list[:,4] 
lb_name = img_list[:,0]
lb_id = img_list[:,1]
a = 0
c = 0



pred_data = []
pred_target = []
pred_name = []
pred_id = []  

posi_data = []
posi_target = []
nega_data = []
nega_target = []    
posi_name = []
posi_id = []  
nega_name = []
nega_id = []      
     
for i in range(len(lb_target)):
    lb_data[i] = main_eval(lb_data[i])
    if(lb_target[i]==0):
        a =a+1
        if(a<20):
            posi_data.append(lb_data[i])
            posi_target.append(lb_target[i])
            posi_name.append(lb_name[i])
            posi_id.append(lb_id[i])            
        else:
            pred_data.append(lb_data[i])
            pred_target.append(lb_target[i])                
            pred_name.append(lb_name[i])
            pred_id.append(lb_id[i])  
for i in range(len(lb_target)):
    lb_data[i] = main_eval(lb_data[i])
    if(lb_target[i]==1):
        c =c+1
        if(c<75):
            nega_data.append(lb_data[i])
            nega_target.append(int(lb_target[i]))
            nega_name.append(lb_name[i])
            nega_id.append(lb_id[i])                  
        else:
            pred_data.append(lb_data[i])
            pred_target.append(lb_target[i])
            pred_name.append(lb_name[i])   
            pred_id.append(lb_id[i])              
lb_num = 10    

for i in posi_id:
    print(lines[i+1],end='',file=f1)

for i in nega_id:
    print(lines[i+1],end='',file=f1)

for i in pred_id:
    print(lines[i+1],end='',file=f1)
f1.close()
f2 = open('train_name.txt','w')
f3 = open('eval_name.txt','w')
for k in range(100):
    if k%10==0:
        print(k)
    np.random.seed(k)            
    np.random.shuffle(posi_data)
    np.random.shuffle(nega_data)
    np.random.shuffle(posi_target)
    np.random.shuffle(nega_target)    
    np.random.shuffle(posi_name)
    np.random.shuffle(nega_name)
    
    train_x = np.append(posi_data[:lb_num],nega_data[:lb_num]).reshape(lb_num*2,len(lb_data[0]))
    train_y = np.append(posi_target[:lb_num],nega_target[:lb_num])
    train_y = np.array(train_y)
    train_name = np.append(posi_name[:lb_num],nega_name[:lb_num])
    
    eval_num = len(posi_data) + len(nega_data) - lb_num*2
    eval_data = np.append(posi_data[lb_num:],nega_data[lb_num:]).reshape(eval_num,len(lb_data[0]))
    eval_target = np.append(posi_target[lb_num:],nega_target[lb_num:])
    eval_target = np.array(eval_target)
    eval_name = np.append(posi_name[lb_num:],nega_name[lb_num:])
    for i in range(len(train_name)):
        print(train_name[i],file=f2)
    for i in range(len(eval_name)):
        print(eval_name[i],file=f3)
        
    b = 0
    for classifier in classifiers: 
        #predict experimental prediction accuracy
        model = classifiers[classifier](train_x, train_y) 
        predict_lb = model.predict(eval_data)     
#        f1[b] = f1_score(eval_target, predict_lb, average='macro')    
#        cf_mat = confusion_matrix(predict_lb, eval_target, normalize='true')
#        f1_p[b] = f1_p[b] + f1[b]
#        recall[b] = (cf_mat[0,0]+cf_mat[1,1])/2
#        recall_p[b] = recall_p[b] + recall[b]
#        b = b + 1
#    print(f1[5])
#    print(k,f1[0],f1[1],f1[2],f1[3],f1[4],f1[5],file=f)

#print(i,f1_p[0]/100.0,f1_p[1]/100.0,f1_p[2]/100.0
#      ,f1_p[3]/100.0,f1_p[4]/100.0,f1_p[5]/100.0,file=f)
f2.close()
f3.close()
f.close()

0
10
20
30
40
50
60
70
80
90
